In [2]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
cities_path = "output_data/cities.csv"

In [4]:
cities_df = pd.read_csv(cities_path)

In [5]:
cities_df

,City,Lat,Lng,Temp,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Geraldton,-28.7667,114.6000,81.0,81.0,74.0,32,0,8.23,AU,1634968588
1,Cherskiy,68.7500,161.3000,-7.0,-7.0,-7.0,100,99,1.44,RU,1634968614
2,Tuatapere,-46.1333,167.6833,60.0,60.0,60.0,69,98,1.48,NZ,1634968615
3,Sitka,57.0531,-135.3300,44.0,44.0,44.0,91,100,3.14,US,1634968615
4,Sobolevo,54.4299,31.8996,42.0,42.0,42.0,79,94,7.08,RU,1634968615
...,...,...,...,...,...,...,...,...,...,...,...
556,Olinda,-8.0089,-34.8553,80.0,80.0,80.0,78,20,2.06,BR,1634968970
557,Yining,29.0361,114.5636,68.0,68.0,68.0,48,18,0.95,CN,1634969231
558,Kamskiye Polyany,55.4307,51.4171,47.0,47.0,47.0,79,4,7.34,RU,1634969231
559,Rosário do Sul,-30.2583,-54.9142,58.0,58.0,58.0,85,44,3.10,BR,1634969232
